# TorchMD API tutorial

## System setup

We use the `moleculekit` library for reading the input topologies and starting coordinates

In [1]:
from moleculekit.molecule import Molecule
import os

testdir = "../test-data/prod_alanine_dipeptide_amber/"
mol = Molecule(os.path.join(testdir, "structure.prmtop"))  # Reading the system topology
mol.read(os.path.join(testdir, "input.coor"))  # Reading the initial simulation coordinates
mol.read(os.path.join(testdir, "input.xsc"))  # Reading the box dimensions

Next we will load a forcefield file and use the above topology to extract the relevant parameters which will be used for the simulation

In [2]:
from torchmd.forcefields.forcefield import ForceField
from torchmd.parameters import Parameters
import torch

precision = torch.float
device = "cuda:0"

ff = ForceField.create(mol, os.path.join(testdir, "structure.prmtop"))
parameters = Parameters(ff, mol, precision=precision, device=device)

Now we can create a `System` object which will contain the state of the system during the simulation, including:
1. The current atom coordinates
1. The current box size
1. The current atom velocities
1. The current atom forces

In [3]:
from torchmd.integrator import maxwell_boltzmann
from torchmd.systems import System

system = System(mol.numAtoms, nreplicas=1, precision=precision, device=device)
system.set_positions(mol.coords)
system.set_box(mol.box)
system.set_velocities(maxwell_boltzmann(parameters.masses, T=300, replicas=1))

Lastly we will create a `Force` object which will be used to evaluate the potential on a given `System` state

In [4]:
from torchmd.forces import Forces

forces = Forces(parameters, cutoff=9, rfa=True, switch_dist=7.5)
# Evaluate current energy and forces. Forces are modified in-place
Epot = forces.compute(system.pos, system.box, system.forces, returnDetails=True)

print(Epot)
print(system.forces)

[{'electrostatics': -2568.498046875, 'lj': 359.2510986328125, 'bonds': 3.957749366760254, 'angles': 2.8445725440979004, 'dihedrals': 10.57987117767334, '1-4': 0.0, 'impropers': 1.2417081594467163, 'external': 0.0}]
tensor([[[  3.0404,   1.7028,   3.8141],
         [-15.2398, -17.4599,   5.3314],
         [  2.5749,   3.8611,  -4.1888],
         ...,
         [-22.4462,   8.8784,  32.4494],
         [  1.1741,  -8.0141, -15.6699],
         [ 20.2039,  -3.2618, -10.9875]]], device='cuda:0')


## Dynamics

For performing the dynamics we will create an `Integrator` object for integrating the time steps of the simulation as well as a `Wrapper` object for wrapping the system coordinates within the periodic cell

In [5]:
from torchmd.integrator import Integrator
from torchmd.wrapper import Wrapper

langevin_temperature = 300  # K
langevin_gamma = 0.1
timestep = 1  # fs

integrator = Integrator(system, forces, timestep, device, gamma=langevin_gamma, T=langevin_temperature)
wrapper = Wrapper(mol.numAtoms, mol.bonds if len(mol.bonds) else None, device)

In [6]:
from torchmd.minimizers import minimize_bfgs

minimize_bfgs(system, forces, steps=500)  # Minimize the system

Iter  Epot            fmax    
   0   -2190.623047    64.694486
   1   -1917.852360    153.854084
   2   -2307.939634    37.247699
   3   -2360.717360    24.187768
   4   -2389.794320    18.115508
   5   -2406.955990    13.061252
   6   -2453.368619    17.315441
   7   -2481.077834    87.203251
   8   -2511.371303    30.950492
   9   -2522.751084    25.995901
  10   -2530.702911    12.662913
  11   -2540.422623    23.033506
  12   -2553.542983    37.350280
  13   -2570.214845    33.395550
  14   -2555.465914    92.351030
  15   -2578.091946    38.600623
  16   -2589.268017    15.880633
  17   -2594.456524    16.902868
  18   -2600.700359    19.452479
  19   -2609.083020    32.326193
  20   -2617.947584    17.985517
  21   -2622.969692    16.233069
  22   -2629.243030    15.250336
  23   -2632.600382    20.044043
  24   -2638.861904    19.499742
  25   -2645.903416    42.060598
  26   -2652.047075    27.136287
  27   -2654.642003    19.016126
  28   -2658.477778    9.471823
  29   -2663

 258   -2821.451533    3.006873
 259   -2821.596806    4.157404
 260   -2821.743748    4.265468
 261   -2821.874253    3.577991
 262   -2822.013755    3.446091
 263   -2822.191061    5.302252
 264   -2822.335139    5.389509
 265   -2822.481897    3.555444
 266   -2822.654975    3.144759
 267   -2822.774793    3.487382
 268   -2822.928122    4.261543
 269   -2823.049058    4.450009
 270   -2823.208372    2.594940
 271   -2823.267165    2.072611
 272   -2823.417901    3.469099
 273   -2823.471073    8.906036
 274   -2823.626930    2.982303
 275   -2823.709906    1.937727
 276   -2823.783084    2.503219
 277   -2823.906724    3.575660
 278   -2824.041201    5.329663
 279   -2824.229882    2.828133
 280   -2824.304781    1.820237
 281   -2824.387959    2.428443
 282   -2824.462311    3.151273
 283   -2824.571830    2.775937
 284   -2824.663102    2.812655
 285   -2824.739449    1.907535
 286   -2824.808585    2.646491
 287   -2824.865301    4.879392
 288   -2824.949253    2.957240
 289   -

In [7]:
from torchmd.utils import LogWriter

logger = LogWriter(path="logs/", keys=('iter','ns','epot','ekin','etot','T'), name='monitor.csv')

Writing logs to  logs/monitor.csv


Now we can finally perform the full dynamics

In [8]:
from tqdm import tqdm 
import numpy as np

FS2NS = 1E-6 # Femtosecond to nanosecond conversion

steps = 1000
output_period = 10
save_period = 100
traj = []

trajectoryout = "mytrajectory.xtc"

iterator = tqdm(range(1, int(steps / output_period) + 1))
Epot = forces.compute(system.pos, system.box, system.forces)
for i in iterator:
    Ekin, Epot, T = integrator.step(niter=output_period)
    wrapper.wrap(system.pos, system.box)
    currpos = system.pos.detach().cpu().numpy().copy()
    traj.append(currpos)
    
    if (i*output_period) % save_period  == 0:
        np.save(trajectoryout, np.stack(traj, axis=2))

    logger.write_row({'iter':i*output_period,'ns':FS2NS*i*output_period*timestep,'epot':Epot,'ekin':Ekin,'etot':Epot+Ekin,'T':T})

100%|██████████| 100/100 [00:25<00:00,  3.87it/s]
